In [ ]:
import requests
import random
import time

from GPMLoginAPI import GPMLoginAPI
from selenium.webdriver.chrome import service
from selenium.webdriver.chrome.options import Options
from UndetectChromeDriver import UndetectChromeDriver


In [ ]:
HEADERS = {"Accept": 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
           "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}


In [ ]:
domains = open('domain_go.txt').read().splitlines()

In [ ]:
len(domains)

In [ ]:
api = GPMLoginAPI('http://127.0.0.1:19995')

In [ ]:
def init_driver():
    profileid = api.Create(time.time())['profile_id']
    startedResult = api.Start(profileid)
    browserLocation = str(startedResult["browser_location"])
    seleniumRemoteDebugAddress = str(startedResult["selenium_remote_debug_address"])
    gpmDriverPath = str(startedResult["selenium_driver_location"])
    options = Options()
    options.debugger_address = seleniumRemoteDebugAddress
    myService  = service.Service(gpmDriverPath)
    return UndetectChromeDriver(service = myService, options=options), profileid

In [ ]:
def reload_cookies():
    driver.get(f'https://www.godaddy.com/domainsearch/find?checkAvail=1&domainToCheck={random.choice(domains)}')
    time.sleep(10)
    return {"bm_sz": driver.get_cookie("bm_sz")['value'],
          "visitor": driver.get_cookie("visitor")['value'],
          "market": "en-US"}

In [ ]:
proxies = {'http': 'http://192.168.113.6:1234',
           'https': 'http://192.168.113.6:1234'}

In [ ]:
data = []
c = 0
driver, profile_id = init_driver()
cookies = reload_cookies()

In [ ]:
for domain in domains:
    if domain in [domain['ClientRequestIn']['DomainName'] for domain in data]:
        continue
    while True:
        try:
            r = requests.get(f'https://www.godaddy.com/domainfind/v1/search/exact?req_id={int(time.time() * 1000)}&key=dpp_search&q={domain}', cookies=cookies, headers=HEADERS)
            print(r.status_code)
            if r.status_code == 429:
                print('IP')
                raise
            elif r.status_code == 403:
                c += 1
                if c >= 3:
                    driver.close()
                    api.Delete(profile_id)
                    driver, profile = init_driver()
                    c = 0
                cookies = reload_cookies()
            elif r.status_code == 200:
                data.append(r.json())
                c = 0
                print(f'{domain} Done!')
                break
        except Exception as e:
            print(c, e)
